<a href="https://colab.research.google.com/github/Ojas-Mehta/RNNResearchSentimentAnalysis/blob/master/SAToy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import torch.nn as nn
import pickle
import json
import pandas as pd
%matplotlib inline

In [0]:

# ########Generate synthetic data
#torch.manual_seed(2109)
# #train_synth.csv, test_synth.csv
# import numpy as np
# import csv
# dataset = []
# dataset_size = 2000
# for i in range(dataset_size):
#     a = np.floor(100*np.random.rand(10)).astype(int)
#     if(((80 <= a) & (a < 89)).sum()>=((90 <= a) & (a < 100)).sum()):
#       label = 1
#     else:
#       label = 0
#     dataset.append([a.tolist(),label])
# with open('/content/drive/My Drive/sentiment/data/val_small.csv', 'w') as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerows(dataset)
# csvFile.close()


In [0]:
#Read data
#Train = 15000, Val = 2000, Test = 3000
train_data = pd.read_csv('/content/drive/My Drive/sentiment/data/train_small.csv', header = None)
val_data = pd.read_csv('/content/drive/My Drive/sentiment/data/val_small.csv', header = None)
test_data = pd.read_csv('/content/drive/My Drive/sentiment/data/test_small.csv', header = None)
train_X_list = train_data[0].tolist()
train_y = train_data[1].tolist()
val_X_list = val_data[0].tolist()
val_y = val_data[1].tolist()
test_X_list = test_data[0].tolist()
test_y = test_data[1].tolist()


In [0]:
#Preprocesssing:String to Numeric
torch.manual_seed(2109)
def stringToList(string):
    string = string[1:len(string)-1]
    try:
        if len(string) != 0: 
            tempList = string.split(", ")
            newList = list(map(lambda x: int(x), tempList))
        else:
            newList = []
    except:
        newList = [-9999]
    return(newList)
train_X_list = [stringToList(x) for x in train_X_list]
train_X = torch.tensor(train_X_list, dtype=torch.long)
val_X_list = [stringToList(x) for x in val_X_list]
val_X = torch.tensor(val_X_list, dtype=torch.long)
test_X_list = [stringToList(x) for x in test_X_list]
test_X = torch.tensor(test_X_list, dtype=torch.long)

In [27]:
len(test_X_list)

3000

In [0]:
# Define hyperparameters
input_dim = 10
output_dim = 2
n_epochs = 150
batch_size = 100
lr = 0.001
hidden_dimension = 8
patience_param = 4

In [0]:
#Get embeddings for data
torch.manual_seed(2109)
embeddings = nn.Embedding(100, input_dim)
train_input = embeddings(train_X)
train_output = torch.tensor(train_y, dtype=torch.long)

val_input = embeddings(val_X)
val_output = torch.tensor(val_y, dtype=torch.long)
test_input = embeddings(test_X)
result = []

In [8]:
# %%javascript
# IPython.OutputArea.auto_scroll_threshold = 9999;
#########Inference from the saved model
batch_size = 15000
def evaluate(tempmodel, dataX, dataY, criterion, batch_size):
    epoch_loss = 0
    epoch_acc = 0
    correct_pred = 0
    tempmodel.eval()
    with torch.no_grad():
        for i in range(0, dataX.shape[0], batch_size):
            batchX, batchy = dataX[i:i+batch_size], dataY[i:i+batch_size]
            batchX = batchX.to(device)
            batchy = batchy.to(device)
            output, hidden = tempmodel(batchX, batch_size = batch_size)          
            loss = criterion(hidden, batchy)

            epoch_loss += loss.item()*batch_size
            _, predicted_labels = torch.max(hidden, 1)
            correct_pred += (predicted_labels==batchy).sum().item()
        accuracy = correct_pred/ len(dataX)*100
        epoch_loss /= len(dataX)   
    return epoch_loss, accuracy
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")
    
    
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, batch_size):
        super(Model, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers


        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x, batch_size):
        
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)

        hidden = hidden.transpose(0,1).contiguous().view(batch_size, -1)
        hidden = self.fc(hidden)
        return out, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        hidden = hidden.to(device)
        return hidden



# Create and Load the model
batch_size = 15000
model = Model(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dimension, n_layers=1, batch_size = batch_size)

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


model.load_state_dict(torch.load('/content/drive/My Drive/sentiment/Model/myModelsmall8.pt'))
model.eval()
model = model.to(device)
#print Train Accuracy
batch_X, batch_y = train_input[0:batch_size], train_output[0:batch_size]
batch_X = batch_X.to(device)
batch_y = batch_y.to(device)
output, hidden = model(batch_X, batch_size = batch_size)
printLoss, printAccuracy = evaluate(model, train_input, train_output, criterion, batch_size)
print(printLoss, printAccuracy)

#print Validation Accuracy
batch_size = 2000
batch_X, batch_y = val_input[0:batch_size], val_output[0:batch_size]
batch_X = batch_X.to(device)
batch_y = batch_y.to(device)
output, hidden = model(batch_X, batch_size = batch_size)
printLoss, printAccuracy = evaluate(model, val_input, val_output, criterion, batch_size)
print(printLoss, printAccuracy)

# #print Single Example hidden states
# batch_size = 1
# # batch_X, batch_y = val_input[3:4], val_output[3:4]
# batch_X = custom_example
# batch_X = batch_X.to(device)
# output, hidden = model(batch_X, batch_size = batch_size)
# print(output)

GPU is available
0.08412731438875198 97.7
0.098625548183918 97.05


NameError: ignored

In [0]:
###Get parameters of the loaded model
ordir = model.state_dict()
B = ordir['rnn.weight_ih_l0'].cpu()
B_bias = ordir['rnn.bias_ih_l0'].cpu()
A = ordir['rnn.weight_hh_l0'].cpu()
A_bias = ordir['rnn.bias_hh_l0'].cpu()
C = ordir['fc.weight'].cpu()
C_bias = ordir['fc.bias'].cpu()

In [0]:
# ##############################Product of parameter and input word vector
# for i in range(100):
#     inputword = embeddings(torch.tensor(i, dtype=torch.long))
#     print(torch.norm(torch.mm(B,inputword.view(10,1))))#default frob. norm
# #     print(torch.sum(torch.mm(InputParam,inputword.view(10,1))+InputBias.view(8,1)))

In [17]:
custom_example1 = embeddings(torch.tensor([[91, 92, 93, 91, 92, 93, 91, 92, 93, 92]], dtype = torch.long))###all negative
custom_example2 = embeddings(torch.tensor([[81, 82, 83, 81, 82, 83, 81, 82, 83, 82]], dtype = torch.long))###all positive
custom_example3 = embeddings(torch.tensor([[21, 2, 3, 11, 22, 33, 41, 52, 63, 72]], dtype = torch.long))###all neutral
custom_example4 = embeddings(torch.tensor([[82, 91, 83, 98, 81, 92, 83, 91, 82, 93]], dtype = torch.long))###alternate

#print Single Example hidden states
batch_size = 1
# batch_X, batch_y = val_input[3:4], val_output[3:4]
print('NORM of hidden state for [91, 92, 93, 91, 92, 93, 91, 92, 93, 92]------')
batch_X = custom_example1
batch_X = batch_X.to(device)
output, hidden = model(batch_X, batch_size = batch_size)
# print(output)
for i in range(10):
  print(torch.norm(output[0][i]))
print('\nNORM--[81, 82, 83, 81, 82, 83, 81, 82, 83, 82]--------')
batch_X = custom_example2
batch_X = batch_X.to(device)
output, hidden = model(batch_X, batch_size = batch_size)
# print(output)
for i in range(10):
  print(torch.norm(output[0][i]))
  batch_X = custom_example1
print('-\nNORM---[21, 2, 3, 11, 22, 33, 41, 52, 63, 72]------')
batch_X = custom_example3
batch_X = batch_X.to(device)
output, hidden = model(batch_X, batch_size = batch_size)
# print(output)
for i in range(10):
  print(torch.norm(output[0][i]))
  batch_X = custom_example1
print('\nNORM----[82, 91, 83, 98, 81, 92, 83, 91, 82, 93]----')
batch_X = custom_example4
batch_X = batch_X.to(device)
output, hidden = model(batch_X, batch_size = batch_size)
# print(output)
for i in range(10):
  print(torch.norm(output[0][i]))

  

print('\nSUM of hidden state----[91, 92, 93, 91, 92, 93, 91, 92, 93, 92]-----')
batch_X = custom_example1
batch_X = batch_X.to(device)
output, hidden = model(batch_X, batch_size = batch_size)
# print(output)
for i in range(10):
  print(torch.sum(output[0][i]))
print('-\nSUM--[81, 82, 83, 81, 82, 83, 81, 82, 83, 82]---')
batch_X = custom_example2
batch_X = batch_X.to(device)
output, hidden = model(batch_X, batch_size = batch_size)
# print(output)
for i in range(10):
  print(torch.sum(output[0][i]))
  batch_X = custom_example1
print('-\nSUM--[21, 2, 3, 11, 22, 33, 41, 52, 63, 72]--------')
batch_X = custom_example3
batch_X = batch_X.to(device)
output, hidden = model(batch_X, batch_size = batch_size)
# print(output)
for i in range(10):
  print(torch.sum(output[0][i]))
  batch_X = custom_example1
print('-\nSUM--[82, 91, 83, 98, 81, 92, 83, 91, 82, 93]------')
batch_X = custom_example4
batch_X = batch_X.to(device)
output, hidden = model(batch_X, batch_size = batch_size)
# print(output)
for i in range(10):
  print(torch.sum(output[0][i]))

NORM of hidden state for [91, 92, 93, 91, 92, 93, 91, 92, 93, 92]------
tensor(2.1899, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.0665, device='cuda:0', grad_fn=<NormBackward0>)
tensor(1.8245, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.3953, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.2119, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.0407, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.4358, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.2721, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.1057, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.4024, device='cuda:0', grad_fn=<NormBackward0>)

NORM--[81, 82, 83, 81, 82, 83, 81, 82, 83, 82]--------
tensor(2.2795, device='cuda:0', grad_fn=<NormBackward0>)
tensor(1.7783, device='cuda:0', grad_fn=<NormBackward0>)
tensor(1.9698, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.5141, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.0058, device='cuda:0', grad_fn=<NormBackward0>)
tensor(2.1564, de